# Imports

In [ ]:
import os
import numpy as np
import pandas as pd

import optuna
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb

# Config

In [ ]:
from config import run_config

run_config()

# High Accuracy Pipeline (XGBoost/CatBoost)

In [ ]:
def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10, log=True),
        "task_type": "GPU" if CatBoostClassifier().get_param("task_type") == "GPU" else "CPU",
        "verbose": False
    }
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    return 1 - model.score(X_val, y_val)

study = optuna.create_study(direction="minimize")
study.optimize(objective_catboost, n_trials=20)

best_model_catboost = CatBoostClassifier(**study.best_params)
best_model_catboost.fit(X_train, y_train)


# High Speed Pipeline (LightGBM)

In [ ]:
def objective_lightgbm(trial):
    params = {
        "objective": "multiclass",
        "num_class": len(set(y_train)),
        "metric": "multi_logloss",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "verbosity": -1
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
    return 1 - model.score(X_val, y_val)

study = optuna.create_study(direction="minimize")
study.optimize(objective_lightgbm, n_trials=20)

best_model_lgb = lgb.LGBMClassifier(**study.best_params)
best_model_lgb.fit(X_train, y_train)